In [72]:
!pip install git+https://github.com/openai/CLIP.git
!pip install ftfy regex tqdm
!pip install torch torchvision

In [73]:
!pip install faiss-cpu

In [74]:
!pip install gradio

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import gradio as gr
import numpy as np
import faiss
import torch
import clip
import os

# Clip

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model, preprocess = clip.load("ViT-B/16", device = device)

def encode(image):
    img = preprocess(image).unsqueeze(0).to(device)

    with torch.no_grad():
        feature = model.encode_image(img)
    feature =feature.cpu().numpy()[0]

    return feature

100%|███████████████████████████████████████| 335M/335M [00:07<00:00, 45.4MiB/s]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Unzip Dataset

In [75]:
!unzip "/content/drive/MyDrive/clip_dataset/fashion_small_224.zip" -d "/content/fashion_small_224"

In [ ]:
data = np.load("/content/drive/MyDrive/clip_dataset/clip_features.npz", allow_pickle=True)
features = data['features'].astype('float32')
paths = data['paths']

In [ ]:
print(paths)

['30080.jpg' '9402.jpg' '57256.jpg' ... '7680.jpg' '12641.jpg' '3045.jpg']


In [ ]:
index = faiss.IndexFlatL2(512)
index.add(features)

In [71]:
def buy(img):
    return Image.fromarray(img)

def show_image(image):

    encode_user = encode(image)
    encode_user = encode_user.reshape(1, -1).astype('float32')
    distances, indices = index.search(encode_user, k=4)
    similar_images = indices[0]
    img1 = Image.open('/content/fashion_small_224/kaggle/working/fashion_small_224/'+ str(paths[similar_images[0]]))
    img2 = Image.open('/content/fashion_small_224/kaggle/working/fashion_small_224/'+ str(paths[similar_images[1]]))
    img3 = Image.open('/content/fashion_small_224/kaggle/working/fashion_small_224/'+ str(paths[similar_images[2]]))
    img4 = Image.open('/content/fashion_small_224/kaggle/working/fashion_small_224/'+ str(paths[similar_images[3]]))

    return np.array(img1), np.array(img2), np.array(img3), np.array(img4)

with gr.Blocks(css=".green-btn {background-color: #4CAF50 !important; color: white;}") as demo:
    gr.Markdown("Upload Image")

    with gr.Row():

        with gr.Column(scale=1):
            image_input = gr.Image(height=800, label="Upload Image", type="pil")
            submit_btn = gr.Button("Submit", variant="primary")
            clear_btn = gr.Button("Clear", variant="stop")

        with gr.Column(scale=1):
            with gr.Row():
                with gr.Column():
                    img1 = gr.Image(height=250, type="numpy")
                    btn_buy1 = gr.Button("BUY$", elem_classes="green-btn")
                with gr.Column():
                    img2 = gr.Image(height=250)
                    btn_buy2 = gr.Button("BUY$", elem_classes="green-btn")
            with gr.Row():
                with gr.Column():
                    img3 = gr.Image(height=250)
                    btn_buy3 = gr.Button("BUY$", elem_classes="green-btn")
                with gr.Column():
                    img4= gr.Image(height=250)
                    btn_buy4 = gr.Button("BUY$", elem_classes="green-btn")



    submit_btn.click(fn=show_image, inputs=image_input, outputs=[img1, img2, img3, img4])
    clear_btn.click(fn=lambda: [None, None, None, None, None], inputs=[], outputs=[image_input, img1, img2, img3, img4])

    btn_buy1.click(fn=buy, inputs=img1, outputs=image_input)
    btn_buy2.click(fn=buy, inputs=img2, outputs=image_input)
    btn_buy3.click(fn=buy, inputs=img3, outputs=image_input)
    btn_buy4.click(fn=buy, inputs=img4, outputs=image_input)

demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://feac83b63a3765eb79.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


------------

------------

-----------

# Data Processing in Kaggle with big Dataset with 44,441 image

In [ ]:
# dataset_path = '/kaggle/input/fashion-product-images-dataset/fashion-dataset/images/'

# images_feature_list = []
# images_path_list = []

# for image in os.listdir(dataset_path):
#     image_path = os.path.join(dataset_path, image)
#     img = Image.open(image_path)

#     feature_image = encode(img)
#     images_feature_list.append(feature_image)
#     images_path_list.append(image_path)

# np.savez("clip_features.npz", features=images_feature_list, paths=images_path_list)

Selecting 10,000 images at random to continue the project in Google Cloud and managing resource limits and cloud limits for Gradio

In [ ]:
# dataset_path = '/kaggle/input/fashion-product-images-dataset/fashion-dataset/images/'
# output_dir = "/kaggle/working/fashion_small_224/"
# os.makedirs(output_dir, exist_ok=True)

# N = 10000

# images_feature_list = []
# images_path_list = []

# all_images = [f for f in os.listdir(dataset_path) if f.lower().endswith((".jpg", ".jpeg", ".png"))]
# print(f"total number of images: {len(all_images)}")

# selected_images = random.sample(all_images, min(N, len(all_images)))

# for img_name in tqdm(selected_images, desc="compressing..."):

#     img_path = os.path.join(dataset_path, img_name)
#     img = Image.open(img_path)
#     img = img.resize((224, 224))

#     feature_image = processing(img)
#     images_feature_list.append(feature_image)

#     save_path = os.path.join(output_dir, os.path.splitext(img_name)[0] + ".jpg")
#     img.save(save_path, format="JPEG", quality=85, optimize=True)

#     images_path_list.append(img_name)


# print(len(images_feature_list))
# print(len(images_path_list))

# np.savez("clip_features.npz", features=images_feature_list, paths=images_path_list)